# Integrating with OPERA's ISCE3 s1-reader

This notebook requires `isce3`, and `s1-reader` to be installed as well as the rest of the packages listed in `requirements.txt`. Install using these commands:
```bash
conda install -c conda-forge isce3
conda install -c conda-forge backoff

git clone https://github.com/opera-adt/s1-reader.git
python -m pip install -e s1-reader
```

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import xml.etree.ElementTree as ET
import datetime

import isce3
import s1reader
from s1reader import Sentinel1BurstSlc
import bursts

In [ ]:
INTERNATIONAL_IDS = {'S1A': ' 2014-016A', 'S1B': '2016-025A'}
zip_path = Path('../example_data/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip')
url_path = 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip'
safe_name = zip_path.with_suffix('.SAFE').name
swath_num = 1
pol = "VV"

orbit_dir = Path('../example_data/sentinel1_orbits')
if not orbit_dir.is_dir():
    orbit_dir.mkdir()

In [ ]:
s1reader.s1_orbit.download_orbit(safe_name,orbit_dir)

## Opera Workflow

In [ ]:
orbit_path = s1reader.get_orbit_file_from_dir(zip_path, orbit_dir)

In [ ]:
# returns the list of the bursts
opera_burst_items = s1reader.load_bursts(zip_path, orbit_path, swath_num, pol)
opera_burst_ids = [x.burst_id for x in opera_burst_items]
opera_burst = opera_burst_items[0]

In [ ]:
opera_burst

In [ ]:
opera_dict = opera_burst.__dict__

## Now with ASF package

In [ ]:
auth = bursts.get_netrc_auth()
manifest, annotations = bursts.edl_download_metadata(url_path, auth)

In [ ]:
slc = bursts.SLCMetadata(url_path, manifest, annotations)
swath = bursts.SwathMetadata(slc, pol, swath_index=swath_num - 1)
asf_burst_list = [bursts.BurstMetadata(swath, x) for x in range(swath.n_bursts)]
asf_burst = asf_burst_list[0]
item = asf_burst.to_stac_item()

In [ ]:
asf_burst = bursts.stac_item_to_opera_burst(item,'VV',orbit_dir)

In [ ]:
asf_dict = asf_burst.__dict__

In [ ]:
[x for x in asf_dict if not opera_dict[x]]

In [ ]:
[x for x in asf_dict if type(asf_dict[x]) != type(opera_dict[x])]

In [ ]:
# attributes in oks don't work correctly when comparing with ==
oks = ['azimuth_fm_rate','doppler','orbit', 'center', 'border','tiff_path']

[x for x in asf_dict if (asf_dict[x] != opera_dict[x]) & (x not in oks)]

Tada! Check out the `example.ipynb` to see how we can download a burst's data directly using the information contained in a burst STAC item.

## Download data with RemoteSentinel1BurstSLC class

In [ ]:
asf_burst = bursts.stac_item_to_opera_burst(item,'VV',orbit_dir, remote=True)

In [ ]:
asf_burst.tiff_path, asf_burst.url_path

In [ ]:
# download into memory
array = asf_burst.edl_download_data()
array.shape

In [ ]:
# download to file
asf_burst.slc_to_file('./test.bin', fmt='ENVI')

In [ ]:
asf_burst.tiff_path

In [ ]:
!gdalinfo test.bin